In [1]:
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import random
import json
import pickle
import numpy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [2]:
with open("data.json") as file:
    data = json.load(file)

In [3]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)

In [4]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return numpy.array(bag).reshape(1,107)

def chat(inp):
    results = model.predict([bag_of_words(inp, words)])
    results_index = numpy.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
        if tg['tag'] == tag:
            responses = tg['responses']

    print(random.choice(responses))

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
model = keras.Sequential()
model.add(keras.Input(shape=(1,len(training[0]))))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(len(output[0]), activation='linear'))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='mse')
model.fit(training, output, batch_size=20, epochs=100, verbose=1)

Epoch 1/100
6/6 [==============================] - 0s 32ms/step - loss: 0.0433
Epoch 2/100
6/6 [==============================] - 0s 18ms/step - loss: 0.0210
Epoch 3/100
6/6 [==============================] - 0s 19ms/step - loss: 0.0184
Epoch 4/100
6/6 [==============================] - 0s 24ms/step - loss: 0.0159
Epoch 5/100
6/6 [==============================] - 0s 17ms/step - loss: 0.0132
Epoch 6/100
6/6 [==============================] - 0s 21ms/step - loss: 0.0112
Epoch 7/100
6/6 [==============================] - 0s 18ms/step - loss: 0.0096
Epoch 8/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0080
Epoch 9/100
6/6 [==============================] - 0s 25ms/step - loss: 0.0066
Epoch 10/100
6/6 [==============================] - 0s 23ms/step - loss: 0.0054
Epoch 11/100
6/6 [==============================] - 0s 14ms/step - loss: 0.0047
Epoch 12/100
6/6 [==============================] - 0s 38ms/step - loss: 0.0036
Epoch 13/100
6/6 [==============================]

6/6 [==============================] - 0s 17ms/step - loss: 3.3865e-04
Epoch 95/100
6/6 [==============================] - 0s 16ms/step - loss: 3.0981e-04
Epoch 96/100
6/6 [==============================] - 0s 15ms/step - loss: 2.8569e-04
Epoch 97/100
6/6 [==============================] - 0s 15ms/step - loss: 2.4957e-04
Epoch 98/100
6/6 [==============================] - 0s 15ms/step - loss: 2.8356e-04
Epoch 99/100
6/6 [==============================] - 0s 15ms/step - loss: 2.5478e-04
Epoch 100/100
6/6 [==============================] - 0s 32ms/step - loss: 2.2168e-04


In [7]:
#model.save_weights('weights')
model.save('mymodel.h5')